<a href="https://colab.research.google.com/github/JohnBortotti/Bayes-Sklearn-test/blob/main/Bayes_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.metrics import accuracy_score, top_k_accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import spacy

import string
import math
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
pt_stopwords = stopwords.words('portuguese')

os.system("python -m spacy download pt_core_news_lg")

spacy_nlp_model = spacy.load('pt_core_news_lg', disable=[
    "tagger", "parser", "ner", "textcat", "entity_linker", 
    "attribute_ruler", "entity_ruler", "morphologizer",
    "senter", "tok2vec", "transformer"
])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# set csv path and separator
csv = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle_databases/tweet_emotions.csv', sep=';')

In [ ]:
# csv config labels, columns, etc...
csv_comment_column = 'content'
csv_label_column = 'sentiment'
csv = csv[[csv_comment_column, csv_label_column]]

In [ ]:
# text processing (remove stopwords, tokenization)
train_dataset = []

bad_words = set(pt_stopwords + list(string.punctuation))

for i, row in csv.iterrows():
  treated_string = ""
  row_tokens = row_tokens = spacy_nlp_model(row[csv_comment_column].lower())
  for token in row_tokens:
    word = token.lemma_
    if not (word in bad_words):
        treated_string += word
  if treated_string != "":
    train_dataset.append((treated_string, row[csv_label_column]))

In [ ]:
# shuffle dataset
train_dataframe = pd.DataFrame(train_dataset)
train_dataframe = train_dataframe.sample(frac=1).reset_index(drop=True)

In [ ]:
# droping some rows to avoid RAM crashing on Google Colab
train_dataframe = train_dataframe[:20000]

In [ ]:
# slice dataframe (train and validation)
total_len = len(train_dataframe)
train_len = math.floor(len(train_dataframe)*7/10)

validation_dataframe = train_dataframe[train_len+1:]
train_dataframe = train_dataframe[0:train_len]

In [ ]:
# labels encoding
label_encoder = preprocessing.LabelEncoder()
train_labels=label_encoder.fit_transform(train_dataframe[1])

# input encoding
vectorizer = CountVectorizer()
train_inputs = vectorizer.fit_transform(train_dataframe[0])

In [ ]:
# training step
model = MultinomialNB()
model.fit(train_inputs.toarray(), train_labels)

MultinomialNB()

In [ ]:
# validation
validation_encoded = vectorizer.transform(validation_dataframe[0]).toarray()
validation_predict = model.predict(validation_encoded)

print('Accuracy:', accuracy_score(label_encoder.transform(validation_dataframe[1]), validation_predict))
print('Test dataset:', len(train_inputs.toarray()))
print('Validation dataset:', len(validation_encoded))

Accuracy: 0.2303717286214369
Test dataset: 14000
Validation dataset: 5999


In [ ]:
# report
print(classification_report(label_encoder.transform(validation_dataframe[1]), validation_predict, target_names=label_encoder.classes_))

              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        14
     boredom       0.00      0.00      0.00        31
       empty       0.00      0.00      0.00       116
  enthusiasm       0.00      0.00      0.00       117
         fun       0.00      0.00      0.00       287
   happiness       0.27      0.03      0.05       791
        hate       0.00      0.00      0.00       185
        love       0.42      0.06      0.10       573
     neutral       0.22      0.91      0.36      1289
      relief       0.00      0.00      0.00       231
     sadness       0.30      0.03      0.06       802
    surprise       0.00      0.00      0.00       358
       worry       0.28      0.10      0.15      1205

    accuracy                           0.23      5999
   macro avg       0.12      0.09      0.06      5999
weighted avg       0.22      0.23      0.13      5999



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
